In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=c5c45eed5f925d71e311f6a0e7b9c26b868dad6dbaa9aecb600da3e7f6afed7f
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import sum, max, col

In [ ]:
# Creating spark session
spark = SparkSession.builder.appName('Covid19').getOrCreate()

In [ ]:
#1) Defining schema
myschema = StructType([ \
    StructField("date", DateType(), True), \
    StructField("county", StringType(), True), \
    StructField("state", StringType(), True), \
    StructField("fips", IntegerType(), True), \
    StructField("cases", IntegerType(), True), \
    StructField("deaths", IntegerType(), True), \
])

In [ ]:
# Reading in data
counties = spark.read.format('csv').option('header', 'true').schema(myschema).load('/content/drive/MyDrive/us-counties.csv')
counties.show(10)
counties.printSchema()

+----------+-----------+----------+-----+-----+------+
|      date|     county|     state| fips|cases|deaths|
+----------+-----------+----------+-----+-----+------+
|2020-01-21|  Snohomish|Washington|53061|    1|     0|
|2020-01-22|  Snohomish|Washington|53061|    1|     0|
|2020-01-23|  Snohomish|Washington|53061|    1|     0|
|2020-01-24|       Cook|  Illinois|17031|    1|     0|
|2020-01-24|  Snohomish|Washington|53061|    1|     0|
|2020-01-25|     Orange|California| 6059|    1|     0|
|2020-01-25|       Cook|  Illinois|17031|    1|     0|
|2020-01-25|  Snohomish|Washington|53061|    1|     0|
|2020-01-26|   Maricopa|   Arizona| 4013|    1|     0|
|2020-01-26|Los Angeles|California| 6037|    1|     0|
+----------+-----------+----------+-----+-----+------+
only showing top 10 rows

root
 |-- date: date (nullable = true)
 |-- county: string (nullable = true)
 |-- state: string (nullable = true)
 |-- fips: integer (nullable = true)
 |-- cases: integer (nullable = true)
 |-- deaths: in

In [ ]:
#2) County with most deaths
counties.select('county', 'state', 'deaths').orderBy('deaths', ascending=False).show(1)

+-------------+--------+------+
|       county|   state|deaths|
+-------------+--------+------+
|New York City|New York| 40267|
+-------------+--------+------+
only showing top 1 row



In [ ]:
#3) County with most cases
counties.select('county', 'state', 'cases').orderBy('cases', ascending=False).show(1)

+-----------+----------+-------+
|     county|     state|  cases|
+-----------+----------+-------+
|Los Angeles|California|2908425|
+-----------+----------+-------+
only showing top 1 row



In [ ]:
#4) Deaths in utah county
counties.select('county', 'state', 'deaths').filter(counties.county == 'Utah').orderBy('cases', ascending=False).show(1)

+------+-----+------+
|county|state|deaths|
+------+-----+------+
|  Utah| Utah|   791|
+------+-----+------+
only showing top 1 row



In [ ]:
#5) Death rate in states
df = counties.groupBy('county', 'state').agg(max('cases').alias('CountyCases'), max('deaths').alias('CountyDeaths'))
df.groupBy('state') \
  .agg(sum('CountyCases').alias('AllCases'), sum('CountyDeaths').alias('AllDeaths')) \
  .withColumn('DeathRate', (col('AllDeaths') / col('AllCases'))) \
  .orderBy('DeathRate', ascending=False).show()

+-------------+--------+---------+--------------------+
|        state|AllCases|AllDeaths|           DeathRate|
+-------------+--------+---------+--------------------+
| Pennsylvania| 2850361|    44816| 0.01572292071074506|
|  Mississippi|  801527|    12509|0.015606461167247017|
|      Alabama| 1304721|    19629|0.015044595741158455|
|      Arizona| 2030944|    30242|0.014890612444262373|
|       Nevada|  745456|    10980|0.014729239552703312|
|      Georgia| 2497983|    36748| 0.01471106889038076|
|     Michigan| 2472824|    36153|0.014620126624458513|
|   New Jersey| 2316898|    33632|0.014515960564513415|
|   New Mexico|  526500|     7627|0.014486229819563153|
|         Ohio| 2725342|    38572|0.014153086108092123|
|     Missouri| 1461727|    20588|0.014084709388278386|
|  Connecticut|  779948|    10926|0.014008626216106716|
|      Indiana| 1717471|    23664|0.013778398587225054|
|    Louisiana| 1276677|    17533| 0.01373330920820223|
|West Virginia|  505532|     6928|0.013704374797

In [ ]:
#6) Counties in utah with the most deaths
counties.filter(counties.state == 'Utah').groupBy('county').agg(max(counties.deaths).alias('CountyDeaths')).orderBy('CountyDeaths', ascending=False).show()


+----------+------------+
|    county|CountyDeaths|
+----------+------------+
| Salt Lake|        1638|
|      Utah|         791|
|Washington|         463|
|     Weber|         443|
|     Davis|         404|
| Box Elder|         124|
|     Cache|         109|
|    Tooele|         102|
|   Unknown|          96|
|      Iron|          92|
|    Uintah|          72|
|   Sanpete|          56|
|  San Juan|          49|
|    Carbon|          46|
|   Millard|          35|
|   Wasatch|          35|
|    Sevier|          31|
|     Emery|          28|
|  Duchesne|          27|
|    Summit|          26|
+----------+------------+
only showing top 20 rows

